In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
# from simpletransformers.classification import ClassificationModel
# from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import ast
import numpy as np
import re

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

from google.colab import files
uploaded = files.upload()

Saving multilabel_classification to multilabel_classification


In [13]:
multi_label_classification = pd.read_csv("/content/multilabel_classification", usecols=["text", "labels"])
multi_label_classification.head()

,text,labels
0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...


In [0]:
# summary_state = summary_state.drop(columns=["State", "Count"])
# summary_state = summary_state.rename(columns={"Summary":"text", "Label":"labels"})
# summary_state = summary_state.reset_index(drop=True)
multi_label_classification = multi_label_classification.reset_index()

In [15]:
separation = int(0.8 * len(multi_label_classification))
train_data = multi_label_classification[:separation]
test_data = multi_label_classification[separation:]
print(len(test_data))
train_data

32


,index,text,labels
0,0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...
...,...,...,...
123,123,From week 40 to 43 (corresponding to October 2...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. ...
124,124,From week 44 to 47 (corresponding to November ...,[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. ...
125,125,From week 48 to 52 (corresponding to December ...,[0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. ...
126,126,The first week of the year 2017 is marked by t...,[0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. ...


In [5]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-q5ekn_ge
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-q5ekn_ge
     |████████████████████████████████| 3.8MB 9.1MB/s 
     |████████████████████████████████| 1.1MB 59.8MB/s 
     |████████████████████████████████| 890kB 58.4MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=675468 sha256=aeb1cd44ef5a543c49fb98327479ebdc190afec9cd815bcc4480d0adb8b2917d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ogt9ccn_/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=865307bef273c52a47672ba5a6219adb64f02b0e32b7a45928d4033308a55a8d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [3]:
#This code does language modeling as the first initial step of fine-tuning on a cusotm WHO report of meningitis
#This is so the the model can learn some of the domain specific knowledge necessary to understand some of the indivial countries' reports
!python3 "run_language_modeling.py"  \
    --output_dir=output \
    --model_type=longformer \
    --model_name_or_path="allenai/longformer-base-4096" \
    --per_device_train_batch_size=1\
    --do_train \
    --train_data_file="finetune.txt" \

usage: run_language_modeling.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                                [--model_type MODEL_TYPE]
                                [--config_name CONFIG_NAME]
                                [--tokenizer_name TOKENIZER_NAME]
                                [--cache_dir CACHE_DIR]
                                [--train_data_file TRAIN_DATA_FILE]
                                [--eval_data_file EVAL_DATA_FILE]
                                [--line_by_line] [--mlm]
                                [--mlm_probability MLM_PROBABILITY]
                                [--block_size BLOCK_SIZE] [--overwrite_cache]
                                --output_dir OUTPUT_DIR
                                [--overwrite_output_dir] [--do_train]
                                [--do_eval] [--do_predict]
                                [--evaluate_during_training]
                                [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
      

In [0]:
device = torch.device("cuda")
from transformers import LongformerModel, LongformerTokenizer, LongformerConfig

In [8]:
tokenizer = LongformerTokenizer.from_pretrained('/content/output')
model = LongformerModel.from_pretrained('/content/output')
model.cuda()

LongformerModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): BertSelfOutpu

In [9]:
torch.cuda.empty_cache()
#note when encoded the start of every token is added with an <s> for classification purposes
input_ids = tokenizer.encode("Hello, my dog is cute")  # Batch size 1
print(tokenizer.decode(input_ids))
print(tokenizer.decode(input_ids[0]))

<s> Hello, my dog is cute</s>
<s>


In [16]:
tokenized = multi_label_classification["text"]\
                                    .apply(lambda x: tokenizer.encode(x, add_special_tokens_tokens=True))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)
#padded according to max_length so we can feed the model in batches - padding works as such, add 0s after the length
#of the original sequence l such that the new length is = max_len = 2530
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


2530


In [17]:
np.array(padded.shape)
#Add masking so that know BERT knows where to attend to => so it can ignore padding
#adds a 1 everwhere where the condition of padded != 0 is satisfied, in other words everywhere there isn't padding
attention_mask = np.where(padded != 0, 1, 0)
#add global attention represented with 2 - for now just add 1 at the classification <s> token
attention_mask[:, [0, -1]] = 2
print(attention_mask)
device = torch.device("cuda")

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)

def create_save_feature_representation():
  first_time = True
  for i in range(0,len(input_ids),2):
    with torch.no_grad():
      predictions, hidden_states = model(input_ids[i:i+2], attention_mask=attention_mask[i:i+2])
      scores = predictions[:,0,:].cpu().numpy()
      if not first_time:
        curr = np.load("lang_features.npy")
        scores = np.concatenate((curr, scores))
      else:
        first_time = False
      np.save("lang_features.npy",scores)

create_save_feature_representation()

[[2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 ...
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]]


In [0]:
features = np.load("lang_features.npy")
#convert the labels into numpy arrays since they are stored as string representations
multi_label_classification["labels"] = multi_label_classification["labels"].apply(lambda x: np.fromstring(x[1:-1], dtype=np.float32, sep=' '))

In [19]:
train_features, test_features = features[:separation], features[separation:]
hidden_size = features.shape[1]

#convert to float datatype that can be then cast to a tensor in PyTorch
labels = np.array([label for label in np.array(multi_label_classification["labels"].values)])
train_labels, test_labels = labels[:separation], labels[separation:]
multi_label_classification.iloc[0]["labels"].shape

(20,)

In [20]:
print(train_labels.dtype)
epochs = 50
neural_network = nn.Sequential(
                    nn.Linear(hidden_size, 1000),
                    nn.ReLU(),
                    nn.BatchNorm1d(1000),
                    nn.Linear(1000, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 600),
                    nn.ReLU(),
                    nn.BatchNorm1d(600),
                    nn.Linear(600, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 200),
                    nn.ReLU(),
                    nn.BatchNorm1d(200),
                    nn.Linear(200, 20),
                    nn.Sigmoid(),
                    )
neural_network.cuda()
dtype = torch.cuda.FloatTensor
neural_network.type = torch.cuda.FloatTensor
train_features, test_features = torch.tensor(train_features).to(device), torch.tensor(test_features).to(device)
train_labels, test_labels = torch.tensor(train_labels).to(device), torch.tensor(test_labels).to(device)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(neural_network.parameters(), lr=0.0001)

for epoch in range(epochs):
  optimizer.zero_grad()
  output = neural_network(train_features)
  loss = criterion(output, train_labels)
  print("Epoch {0} with training loss: {1}".format(epoch, loss))
  loss.backward()
  optimizer.step()    

float32
Epoch 0 with training loss: 0.8406721353530884
Epoch 1 with training loss: 0.8182004690170288
Epoch 2 with training loss: 0.806324303150177
Epoch 3 with training loss: 0.7982347011566162
Epoch 4 with training loss: 0.7923925518989563
Epoch 5 with training loss: 0.787848711013794
Epoch 6 with training loss: 0.7841944098472595
Epoch 7 with training loss: 0.7809910774230957
Epoch 8 with training loss: 0.7782623171806335
Epoch 9 with training loss: 0.7758951783180237
Epoch 10 with training loss: 0.7737647294998169
Epoch 11 with training loss: 0.7718138694763184
Epoch 12 with training loss: 0.7699738144874573
Epoch 13 with training loss: 0.7682682275772095
Epoch 14 with training loss: 0.7667174339294434
Epoch 15 with training loss: 0.7653347849845886
Epoch 16 with training loss: 0.7640621662139893
Epoch 17 with training loss: 0.7628525495529175
Epoch 18 with training loss: 0.7617048025131226
Epoch 19 with training loss: 0.7606009244918823
Epoch 20 with training loss: 0.7595781087875

In [21]:
def convert_to_thresholds(data):
  return data>0.5

def test_model():
  preds = neural_network(test_features).cpu().data.numpy()
  correct = test_labels.cpu().numpy()
  print(correct)
  preds = convert_to_thresholds(preds)
  print("F1 Score: ", f1_score(preds, correct, average="micro"))
  print("ROC-AUC: ", roc_auc_score(correct, preds))

test_model()

[[0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0.]
 [1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1.

##Tests on simplified 2 state binary classification problem
**Initial results tested with 20 labels:**
F1 Score: 0.53
ROC-AUC: 0.51

**Tested on only the most present 8 labels** - as done with the forecasting pipeline (to check if this is a signal to noise ratio)
F1 Score: 0.558
ROC-AUC: 0.501

So the issue here isn't due to noise - the Longformer is just not able to learn a meaningful enough feature representation that can map to the labels.

Let's try to finetune our Longformer model first on some domain-specific text - reports from the WHO that outline how they plan to defeat Meningitis by 2030. After finetuning on this data, let's try to regenerate our feature representations of our report text and see if that changes our results with the multilabel classifier

**After training a pre-trained model to do language modeling**
F1 Score: 0.5329
ROC-AUC: 0.566

The results did improve but not significantly

In [0]:
#As input to finetune model - need to tokenize and feed in sentences into longformer
#In order to feed text into the longformer - need to pass in 3 things:
#1. input_ids which are the tokenized sentences (with the correct tags e.g. <s>)
#2. input_masks telling model where to pay attention to - i.e. where to ignore padding etc.
#3. token type ids which tell models about relationship of different sentences (e.g. relevant to question and answering datasets) 
# since our input consists of single sentences, we don't need it however

def tokenize_sentences(sentences):
  input_ids, attention_masks = [], []
  for sentence in sentences:
    #encodings is a dictionary with the encoded sequence and additional information like the attention mask
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=4096, pad_to_max_length=True, return_attention_mask=True)
    input_ids.append(encoded_dict["input_ids"])
    attention_masks.append(encoded_dict["attention_mask"])
  return input_ids, attention_masks

with open("finetune.txt", "r") as f:
  corpus = f.read().strip()
  corpus = corpus.replace('."', '".')
  #split the corpus into an array of sentences - each sentence will be its own item in the array
  sentences = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", corpus)
  sep = int(0.8*len(sentences))
  #save the last 20% of the data for test
  train_dataset, test_dataset = sentences[:sep], sentences[sep:]

input_ids, attention_masks = tokenize_sentences(train_dataset)
print(len(input_ids)*4096)

train_dataset, test_dataset = map(tokenize_sentences, train_sentences), map(tokenize_sentences, test_sentences)
print(train_dataset)

# config = LongformerConfig(dropout=0.2, attention_dropout=0.2)
# config.output_hidden_states = False
#note we're gonna be using the Longformer model we defined above so we don't need to define a new model here

torch.cuda.empty_cache()